In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
def string_to_list(column):
    return [element.strip(" ") for element in column.lower().strip("[]").replace("'", "").replace('"', "").split(", ")]

def single_list_string_to_string(column):
    try:
        return column.replace("'", "").replace('"', "").replace("[", "").replace("]", "").capitalize()
    except:
        return column

In [5]:
PATH = 'data/'

recensement = pd.read_excel(PATH + '1832_cleaned_data_recensement.xlsx')
recensement = recensement[recensement["VOCATION."] != "[]"]
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].apply(string_to_list)
recensement["NOMS DES RUES."] = recensement["NOMS DES RUES."].apply(single_list_string_to_string)
recensement["VOCATION."] = recensement["VOCATION."].apply(single_list_string_to_string)
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].apply(lambda x : x if len(x[0]) > 0 else None)
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].fillna(method="ffill")
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."].apply(single_list_string_to_string)
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."].apply(single_list_string_to_string)

In [6]:
recensement.head(2)

,Unnamed: 0,NOMS DES RUES.,N° des MAISONS.,PROPRIÉTAIRES DES MAISONS.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE.,Année de la naissance. [h],NOMS DE LEURS ÉPOUSES.,Année de la naissance. [f],Noms de leurs enfans actuellement dans la Commune.,...,Date de leur demeure dans la Commune.,VOCATION.,N° de leur Récépissé. [famille],"NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE BAPTÊME.","NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE FAMILLE.","D'où ils sont bourgeois, ou à ce défaut d'où ils sont originaires. [dom/pens]","Indiquer ici s'ils sont Pensionnaires, Commis, Apprentis, Ouvriers ou Domestiques.",N° de leur Récépissé. [dom/pens],OBSERVATIONS.,Page
0,0,Martherey,['7'],[gilliéron],Jean louis,"Deprez, devez",['1768'],"[['Echs', 'Ochs']]",['1793'],"[['Laura', 'Lucia', 'Lucy']]",...,['1831'],Journalier,[],[],[],[],[],[],[],3
1,1,Martherey,['7'],[gilliéron],"Jean andre, jean marc, jean marie, jeanine, je...",Fontannaz,['1778'],"['Sophprant', 'Loretti']",['1778'],"['Jeanne', 'Jeanne', 'Abram', 'Jeannette', ['D...",...,['1812'],Journalier,['1525'],[],[],[],[],[],[],3


In [7]:
global_job_frequencies = recensement["VOCATION."].value_counts().apply(lambda c : c/len(recensement))
global_job_frequencies

Journalier       0.058963
Vigneron         0.043036
Rentier          0.038631
Rentière         0.037953
Rentiere         0.033548
                   ...   
Garde foret      0.000339
Shupoimeus       0.000339
Bonnetier        0.000339
Retraitée        0.000339
Bre de bierre    0.000339
Name: VOCATION., Length: 475, dtype: float64

In [31]:
top_10_streets = list(recensement["NOMS DES RUES."].value_counts()[:15].index)
top_10_streets

['Grand saint jean',
 'Martherey',
 'Rue du pré',
 'De bourg',
 'Cheneau bourg',
 'Cité derrière',
 'Barre',
 'Ouchy',
 'Halle',
 'Saint pierre',
 'Saint laurent',
 'Etraz',
 'Rue de saint françois',
 'Cité devant',
 'Rue du pont']

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=8, figsize=(18,40), )
for street, ax in zip(top_10_streets, axes.flat):
    top_10_street_jobs = list(recensement[recensement["NOMS DES RUES."] == street]["VOCATION."].value_counts()[:15].index)
    top_10_street_values = list(recensement[recensement["NOMS DES RUES."] == street]["VOCATION."].value_counts()[:15].values)
    top_10_street_frequencies = [v/len(recensement[recensement["NOMS DES RUES."] == street]) for v in top_10_street_values]
    top_10_global_frequencies = global_job_frequencies[top_10_street_jobs]
    data = pd.DataFrame(data={"Métier": top_10_street_jobs, "fréquence dans la rue {}".format(street): top_10_street_frequencies, "fréquence dans le recensement": top_10_global_frequencies})
    data = data.melt(id_vars='Métier')
    sns.barplot(data=data, x="Métier", y="value", hue="variable",palette="Blues_d", ax=ax)
    ax.set_title(street, fontsize=18)
    ax.set_ylabel("Fréquence", fontsize=14)
    ax.set_xlabel("Métier", fontsize=14)
    ax.set_xticklabels(top_10_street_jobs, rotation=45)
plt.tight_layout(pad=0.1, w_pad=0.5, h_pad=0.5)
fig.delaxes(axes[7][1])